In [2]:
from tile_match_gym.board import Board
from copy import deepcopy
import numpy as np
from itertools import product
from multiprocessing import Pool

In [62]:
x = product([1, 2, 3], repeat=16)
x = list(x)



In [63]:
len(x)

43046721

In [64]:
def process_combination(combination):
    count_s, count_s_, count_m = 0, 0, 0
    b = Board(4, 4, 3,[], [])
    b.board = np.array(combination).reshape(4, 4)
    no_poss_move = not b.possible_move()
    has_match = len(b.get_colour_lines()) > 0
    if has_match or no_poss_move:
        count_s = 1
    if not has_match and no_poss_move:
        count_s_= 1
    if has_match:
        count_m = 1

    return count_s, count_s_, count_m

In [65]:
num_invalid_s, num_valid_states_no_move, num_states_w_move_and_match = 0, 0, 0
with Pool() as pool:
    results = pool.map(process_combination, x)

    # Aggregate results from parallel processes

    for result in results:
        num_invalid_s += result[0]
        num_valid_states_no_move += result[1]
        num_states_w_move_and_match += result[2]

print(len(x) - num_invalid_s, num_valid_states_no_move, num_states_w_move_and_match)

8426718 57420 34562583


In [45]:
len(x)

531441

In [41]:
count_s

13303

In [33]:
count_s_

6377

In [34]:
count_m

2458

In [20]:
count_poss

6380

In [21]:
for i, combination in enumerate(x):
    b = Board(3, 3, 3, ["vertical_laser"], [])
    b.board = np.array(combination).reshape(3,3)
    no_poss_move = not b.possible_move()
    has_match = len(b.get_colour_lines()) > 0
    if has_match or no_poss_move:
        count_tot -= 1
    if has_match:
        count_m += 1
    if not no_poss_move:
        count_poss += 1

    # if i % 1000 == 1:
    #     print(i, count_m / i, count_poss / i, count_tot, b.board)

    
        


    

In [7]:
valid_arr

10077696

In [ ]:

board = Board(10, 10, 5)  # cookie , and vertical laser, horizontal laser, bomb
board.board = np.array([[4, 4, 5, 4, 2, 2, 6, 6, 3, 3],
                        [3, 6, 3, 6, 3, 4, 5, 4, 2, 2],
                        [6, 5, 6, 4, 6, 3, 4, 5, 2, 3],
                        [2, 4, 6, 2, 3, 4, 6, 3, 4, 3],
                        [2, 5, 2, 3, 4, 4, 2, 6, 5, 6],
                        [4, 5, 3, 4, 6, 3, 5, 2, 5, 6],
                        [4, 4, 3, 2, 4, 5, 4, 5, 3, 5],
                        [5, 6, 4, 2, 5, 4, 6, 4, 3, 2],
                        [4, 5, 5, 6, 3, 4, 6, 3, 5, 6],
                        [4, 4, 5, 4, 6, 5, 2, 2, 4, 6]])

old_board = deepcopy(board.board)
assert board.check_move_validity((1,1), (1, 2))
assert board.check_move_validity((1,1), (2, 1))
assert np.array_equal(board.board, old_board)